In [1]:
import os
import sys
import copy
sys.path.append('..')

#import models
import random
import numpy as np
import FairCertModule
from FairCertModule import ACS_categories
import folktables
from folktables import ACSDataSource, ACSIncome, ACSEmployment
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn import functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import models, transforms
from FullyConnected import FullyConnected
import pytorch_lightning as pl

SEED = 0
torch.manual_seed(SEED)
random.seed(SEED)
np.random.seed(SEED)

dataset = "Credit"

# Data loaders
import folk_utils
if(dataset in ["Employ", "Folk", "Insurance", "Coverage"]):
    X_train, X_test, X_val, y_train, y_test, y_val, lp_epsilon, sr_epsilon = folk_utils.get_dataset(dataset)
    f_epsilon = lp_epsilon
elif(dataset in ["Adult", "Credit", "German"]):
    X_train, X_test, X_val, y_train, y_test, y_val, lp_epsilon, sr_epsilon = folk_utils.get_UCI_dataset(dataset)
    f_epsilon = lp_epsilon

/Users/matthewwicker/Desktop/Development/CertificationOfDistributionalIF/Folktables/folk_utils.py:976: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features[cf] = features[cf].astype(str).astype('category')


Raw cols: 
(24000, 23)
['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10', 'x11', 'x12', 'x13', 'x14', 'x15', 'x16', 'x17', 'x18', 'x19', 'x20', 'x21', 'x22', 'x23']
Raw cols: 
(6000, 23)
['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10', 'x11', 'x12', 'x13', 'x14', 'x15', 'x16', 'x17', 'x18', 'x19', 'x20', 'x21', 'x22', 'x23']
DATA SHAPES: 
(24000, 145)
(6000, 145)
(6000, 145)
****


/Users/matthewwicker/opt/anaconda3/envs/FairnessEnv/lib/python3.7/site-packages/sklearn/decomposition/_truncated_svd.py:234: RuntimeWarning: invalid value encountered in true_divide
  self.explained_variance_ratio_ = exp_var / full_var


In [2]:
print(y_train.shape)
print(y_val.shape)
print(y_test.shape)
print(len(lp_epsilon))

torch.Size([24000])
torch.Size([6000])
torch.Size([6000])
144


In [3]:
mode = "FAIR-DRO"

if(mode == "FAIR-GLOB"):
    lr = 0.001 #-10
    eps = 1.0
    alpha = 0.02#1
    bs = 256 
    glob_adv = 100
if(mode == "FAIR-IBP"):
    lr = 0.001
    eps = 0.02
    alpha = 0.02#2
    bs = 256 
    glob_adv = 0
elif(mode == "FAIR-IBPG"):
    lr = 0.001
    eps = 0.075
    alpha = 0.02
    bs = 256 
    glob_adv = 0
elif(mode == "FAIR-PGD"):
    lr = 0.001
    eps = 0.02
    alpha = 0.02
    bs = 256
    glob_adv = 0
elif(mode == "FAIR-DRO"):
    lr = 0.001
    eps = 0.02
    alpha = 0.02
    bs = 256
    glob_adv = 16
elif(mode == "SGD"):
    lr = 0.001
    eps = 0.00
    alpha = 0.0
    bs = 128
    glob_adv = 0
if(dataset == "German" and mode != "SGD"):
    alpha = 1.5
    eps *= 6
    #eps *= 3.5 
class custDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.Tensor(X).float()
        self.y = y
        self.transform = transforms.Compose([transforms.ToTensor()])

    def __len__(self):
        return self.X.shape[0]
        
    def __getitem__(self, index):
        return self.X[index], self.y[index]
    

CustTrain = custDataset(X_train, y_train)  
CustVal = custDataset(X_val, y_val) 
CustTest = custDataset(X_val, y_val)

class CustomDataModule(pl.LightningDataModule):
    def __init__(self, train, val, test, batch_size=bs):
        super().__init__()
        self.train_data = train
        self.val_data = val
        self.test_data = test
        self.batch_size = batch_size
        
    def train_dataloader(self):
        return DataLoader(self.train_data, batch_size=self.batch_size)

    def val_dataloader(self):
        return DataLoader(self.val_data, batch_size=self.batch_size)

    def test_dataloader(self):
        return DataLoader(self.test_data, batch_size=self.batch_size)
    
dm = CustomDataModule(CustTrain, CustVal, CustVal)


In [4]:

print(np.round(lp_epsilon, 2))
print(len(lp_epsilon))
print()
#print(np.round(sr_epsilon, 2))

#print(np.round(ex_epsilon, 2))


[1.86 2.97 2.79 2.28 2.06 1.72 1.74 0.31 0.08 0.47 0.39 0.3  0.46 2.3
 2.87 2.02 2.1  1.96 1.61 0.19 1.15 1.51 0.66 2.79 2.06 1.97 2.03 2.
 0.84 0.88 1.02 1.05 0.94 0.36 1.38 1.82 0.89 0.37 1.07 0.84 0.94 2.38
 2.38 0.4  0.67 3.17 3.53 2.36 2.7  1.11 1.43 1.65 0.53 0.36 0.25 0.34
 0.94 0.81 0.51 0.27 0.93 1.77 0.84 0.61 0.69 0.75 0.1  0.77 0.98 0.81
 0.98 0.57 2.09 0.72 1.39 1.14 0.96 1.33 1.85 0.67 2.19 1.32 1.4  0.7
 0.94 1.42 0.89 0.91 1.12 1.13 0.87 0.5  0.11 0.06 0.52 0.16 0.23 0.43
 0.52 0.66 1.93 4.38 2.07 0.51 1.57 2.18 1.46 1.4  0.55 0.32 0.59 2.79
 4.54 2.72 0.63 2.86 1.15 1.88 0.9  0.71 0.59 0.43 2.37 4.36 2.66 0.94
 2.42 1.98 1.18 0.46 0.46 0.89 0.94 2.56 3.48 2.47 0.66 2.05 2.57 1.44
 0.8  0.32 1.51 0.94]
144



In [5]:
import time
if(mode == "FAIR-GLOB" or mode == "FAIR-IBPG"):
    mode_id = "FAIR-IBP"
else:
    mode_id = mode
    
if(dataset == "Coverage"):
    lr *= 2
DEPTH = 2
WIDTH = 2048
#model = FullyConnected(hidden_lay=2, hidden_dim=256,
model = FullyConnected(hidden_lay=DEPTH, hidden_dim=WIDTH,
                       learning_rate = lr, mode=mode_id, 
                       epsilon=eps, alpha=alpha,
                       glob_advs=glob_adv, dataset=dataset)

model.set_fair_interval(lp_epsilon)
# used to be 30, 35
model.MAX_EPOCHS = 5
start = time.time()
trainer = pl.Trainer(max_epochs=10, accelerator="cpu", devices=1)
trainer.fit(model, datamodule=dm)
result = trainer.test(model, datamodule=dm)
end = time.time()

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name | Type       | Params
------------------------------------
0 | lays | ModuleList | 4.5 M 
1 | l1   | Linear     | 296 K 
2 | lf   | Linear     | 4.1 K 
------------------------------------
4.5 M     Trainable params
0         Non-trainable params
4.5 M     Total params
17.990    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Updated eps:  0.004


/Users/matthewwicker/opt/anaconda3/envs/FairnessEnv/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:245: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,
/Users/matthewwicker/opt/anaconda3/envs/FairnessEnv/lib/python3.7/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Torchmetrics v0.9 introduced a new argument class property called `full_state_update` that has
                not been set for this class (_ResultMetric). The property determines if `update` by
                default needs access to the full metric state. If this is not the case, significant speedups can be
                achieved and we recommend setting this to `False`.
                We provide an checking function
 

Training: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Updated eps:  0.02


/Users/matthewwicker/opt/anaconda3/envs/FairnessEnv/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:245: PossibleUserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.7804362177848816     │
└───────────────────────────┴───────────────────────────┘

In [6]:
print("Time: ", end - start)
print("Est: ", (end - start)*30)

Time:  1988.6429069042206
Est:  59659.28720712662


In [7]:

directory = "%sModels"%(dataset)
if not os.path.exists(directory):
    os.makedirs(directory)
MODEL_ID = mode + "%s_%s"%(DEPTH, WIDTH)
print(MODEL_ID)

trainer.save_checkpoint("%s/%s.ckpt"%(directory, MODEL_ID))
torch.save(model.state_dict(), "%s/%s.pt"%(directory, MODEL_ID))




FAIR-DRO2_2048


In [8]:
from folk_utils import evaluate_accuracy, evaluate_delta_PGD, evaluate_delta_IBP
from folk_utils import compute_DIF_certification
lp_epsilon = torch.Tensor(lp_epsilon)
X_test = torch.Tensor(X_test)
y_test = torch.Tensor(y_test)
ibp = evaluate_delta_IBP(model, X_test, y_test, lp_epsilon, 0.025, 2)
print(ibp)

0.00038952514


In [9]:

dro1, ibp_trend, vg = compute_DIF_certification(model, lp_epsilon, 0.075, 0.025, X_test, y_test, lr = 0.1, N=200, iters=1000, rettrend=True)
print(dro1)
dro1, ibp_trend, vg = compute_DIF_certification(model, lp_epsilon, 0.075, 0.025, X_test, y_test, lr = 10, N=200, iters=1000, rettrend=True)
print(dro1)
dro1, ibp_trend, vg = compute_DIF_certification(model, lp_epsilon, 0.075, 0.025, X_test, y_test, lr = 1000, N=200, iters=1000, rettrend=True)
print(dro1)


/Users/matthewwicker/Desktop/Development/CertificationOfDistributionalIF/Folktables/folk_utils.py:626: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1659484744261/work/torch/csrc/utils/tensor_new.cpp:204.)
  v = torch.Tensor([f_epsilon.numpy() for i in range(N)]).float()
100%|███████████████████████████████████████| 1000/1000 [01:42<00:00,  9.78it/s]


0.0511191263794899


100%|███████████████████████████████████████| 1000/1000 [01:38<00:00, 10.11it/s]


0.1571820080280304


100%|███████████████████████████████████████| 1000/1000 [01:41<00:00,  9.83it/s]

0.15606330335140228


In [10]:
from folk_utils import compute_DIF_falsification
dro1, ibp_trend = compute_DIF_falsification(model, lp_epsilon, 0.075, 0.025, X_test, y_test, lr = 0.1, N=200, iters=1000, rettrend=True)
print(dro1)
dro1, ibp_trend = compute_DIF_falsification(model, lp_epsilon, 0.075, 0.025, X_test, y_test, lr = 10, N=200, iters=1000, rettrend=True)
print(dro1)
dro1, ibp_trend = compute_DIF_falsification(model, lp_epsilon, 0.075, 0.025, X_test, y_test, lr = 1000, N=200, iters=1000, rettrend=True)
print(dro1)

 13%|█████▎                                  | 132/1000 [00:13<01:27,  9.96it/s]
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/matthewwicker/opt/anaconda3/envs/FairnessEnv/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/wk/dw_2kpzx6yggmhdyw5k5gj200000gn/T/ipykernel_23236/2165531218.py", line 2, in <module>
    dro1, ibp_trend = compute_DIF_falsification(model, lp_epsilon, 0.075, 0.025, X_test, y_test, lr = 0.1, N=200, iters=1000, rettrend=True)
  File "/Users/matthewwicker/Desktop/Development/CertificationOfDistributionalIF/Folktables/folk_utils.py", line 717, in compute_DIF_falsification
    eps.mean().backward()
  File "/Users/matthewwicker/opt/anaconda3/envs/FairnessEnv/lib/python3.7/site-packages/torch/_tensor.py", line 396, in backward
    torch.autograd.backward(self, gradient, retain_graph, create_graph, inputs=inputs)
  File "/Users/matthewwicker/opt/anaconda3/envs/FairnessEnv/lib/python3.7/site-packages/torch/autograd/__init__.py", line 175,

TypeError: object of type 'NoneType' has no len()

In [ ]:
import matplotlib 
import matplotlib.pyplot as plt

plt.plot(ibp_trend)

In [ ]:

from FairCertModule import *
def global_pgd_ibp(model, vec, eps, npoints=128, iterations=1000, lr=0.1, nclasses=2):
    points = torch.randn((npoints, model.in_dim))
    points.requires_grad_()
    lab = torch.ones((npoints)).long()
    vec = torch.Tensor(vec)
    for i in range(iterations):
        points.requires_grad_()
        with torch.enable_grad():
            weights = [t for t in model.parameters()]
            logit_l, logit_u = interval_bound_forward(model, weights, points, vec, eps)
            v1 = torch.nn.functional.one_hot(lab, num_classes=nclasses)
            v2 = 1 - torch.nn.functional.one_hot(lab, num_classes=nclasses)
            min_logit = torch.add(torch.multiply(v2, logit_u), torch.multiply(v1, logit_l))
            max_logit = torch.add(torch.multiply(v1, logit_u), torch.multiply(v2, logit_l))
            min_i_softmax = F.softmax(min_logit, dim=-1)
            max_i_softmax = F.softmax(max_logit, dim=-1)
            delta = torch.abs((max_i_softmax - min_i_softmax))
            loss = torch.mean(delta)
        grad = torch.autograd.grad(loss, [points])[0]
        #print("Grad: ", grad)
        points = points + (lr *  torch.sign(grad.detach()))
        points = torch.clip(points, -2.5, 2.5)
    print(torch.mean(delta))
    return points

global_worst_case = global_pgd_ibp(model, lp_epsilon, 0.05)


In [ ]:
# New 2 layer experiments

two_layer_widths = [8, 128, 1024, 2048]

credit_dro_upper = [0.119, 0.224, 0.241, 0.2560]

adult_dro_upper = [0.0325, 0.23517, 0.29682, 0.245]

german_dro_upper = [0.03777, 0.2126, 0.6155, 0.8273]

In [ ]:
sizes = [8, 16, 32, 256, 512, 1024, 2048, 4096]
adult_times = [43.011, 47.880, 49.758, 61.686, 70.374, 86.382, 116.154, 178.167] 
credit_times = [24.720, 25.416, 25.970, 37.228, 44.697, 60.636, 82.659, 126.106] 
german_times = [2.070, 2.1468, 2.1720, 2.4902, 2.6261, 2.840, 3.6319, 4.9928]

adult_dro = [213.914, 226.164, 238.306, 363.283, 465.786, 626.370, 930.686, 1617.153]
credit_dro = [164.447, 167.181, 178.785, 284.828, 378.492, 526.416, 833.825, 1456.622]
german_dro = [10.8215, 10.200, 10.495, 14.657, 15.446, 18.594, 31.932, 45.786]

In [ ]:
widths = [8, 32, 256, 512, 1024]
credit_accs = [0.7804, 0.7804, 0.7804, 0.7804]
credit_ibps = [0.0018, 0.0013, 0.0012, 0.0012, 0.00026]
credit_difs = [0.0233, 0.0232, 0.0158, 0.0141, 0.0021]

depths = [1, 2, 3, 4]
depth_ibps = [0.0012, 0.0004, 1.1291529e-06, 4.1245717e-06]
depth_difs = [0.0158, 0.0567, 0.0855, 0.2583]

In [ ]:
# Adult Results
widths = [8, 256, 512, 1024]
ibps = [0.00057, 7.5408557e-06, 5.7115263e-05]
difs = [0.0318, 0.023, 0.0023, 0.0028]

depths = [1, 2, 3, 4]
ibps = [9.383e-05, 1.0150286e-05, 1.0150286e-05]
difs = [0.0142, 0.2426, 0.4764, 0.822]

In [ ]:
# German results
depths = [1,2,3,4]
difs = [0.0932, 0.3653, 0.4995, 0.6598]

widths = [8, 256, 512, 1024]
difs = [0.0994, 0.0552, 0.0282, 0.0234]